# Import Libraries

In [1]:
import tensorflow as tf
assert float(tf.__version__[:3]) >= 2.3

import os
import numpy as np
import glob
import time
import matplotlib.pyplot as plt
import time
from tensorflow.keras.preprocessing import image

# from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

# Load dataset

In [2]:
class_names = ['construction_danger', 'construction_safe', 'park_danger', 'park_safe', 'road_danger', 'road_safe']
label_list = []
image_list = []
dataset_list = []

for label in class_names:
    img_path_list = glob.glob(f'../Test_data/{label}/*')
    
    for i in range(len(img_path_list)):
        image = tf.io.read_file(img_path_list[i])
        image = tf.io.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [224, 224])
        # dataset_list.append(tf.expand_dims(image, 0))
        dataset_list.append(tf.dtypes.cast(tf.expand_dims(image, 0), tf.uint8))
        # _image = tf.expand_dims(image, axis=0)
        # _image = tf.dtypes.cast(_image, tf.uint8)
        # _image = np.array(_image, dtype=np.uint8)
        # dataset_list.append(_image)
        image_list.append(image)
        label_list.append(label)
        
len(image_list)

60

# compiled model test

In [3]:
import tflite_runtime.interpreter as tflite

class Saved_model_test:
    def __init__(self, saved_model_dir, class_names):
        self.saved_model_dir = saved_model_dir
        
        start_time = time.time()
        interpreter = tflite.Interpreter('custom_saved_model_edgetpu.tflite' , 
                          experimental_delegates=[tflite.load_delegate('libedgetpu.so.1')])
        end_time = time.time()
        self.interpreter = interpreter
        self.interpreter.allocate_tensors()
        term = end_time - start_time
        self.loading_term = term
        
        self.input_details = interpreter.get_input_details()
        self.output_details = interpreter.get_output_details()
        
        self.class_names = class_names

        self.saved_model_infer = {}
        self.infer_result = []
    
    def check_speed(self, input):
        N_warmup_run = 50
        N_run = 20
        elapsed_time = []
        
        for i in range(N_warmup_run):
            self.interpreter.set_tensor(self.input_details[0]['index'], input)
            self.interpreter.invoke()
        
        for i in range(N_run):
            start_time = time.time()
            for j in range(N_warmup_run):
                self.interpreter.set_tensor(self.input_details[0]['index'], input)
                self.interpreter.invoke()
            end_time = time.time()
            elapsed_time = np.append(elapsed_time, end_time - start_time)
        self.elapsed_time = elapsed_time
        
        total = 0
        for i in range(N_run):
            total += elapsed_time[i]
        # total /= len(elapsed_time)
        self.speed = (N_run * N_warmup_run)/total
        
    
    def test(self, dataset_list, label_list):
        
        result_list = []
        for i in range(len(dataset_list)):
            self.interpreter.set_tensor(self.input_details[0]['index'], dataset_list[i])
            self.interpreter.invoke()
            preds = self.interpreter.get_tensor(self.output_details[0]['index'])
            result_list.append([label_list[i], self.class_names[np.argmax(preds[0], axis=0)],  preds])
        
        self.result_list = result_list
        
    def calc_accuracy(self):
        self.accuracy = {}
        
        c = np.array(self.result_list)
        for label in self.class_names:
            first_idx = np.where(c[0:, 0:1] == label)[0][0]
            last_idx = np.where(c[0:, 0:1] == label)[0][-1]

            label_len = len(np.where(c[first_idx:last_idx+1, 0:1] == label)[0])
            correct = len(np.where(c[first_idx:last_idx+1:, 1:2] == label)[0])
            self.accuracy[label] = correct/label_len

In [4]:
def test(input, class_names):
    model_test = Saved_model_test(input, class_names)
    print(f'loading term (sec): {model_test.loading_term}')
    
    model_test.check_speed(dataset_list[0])
    print(f'images per sec: {model_test.speed}')

    model_test.test(dataset_list, label_list)
    model_test.calc_accuracy()
    print('accuracy: ')
    print(model_test.accuracy)

## custom_saved_model_edgetpu

In [5]:
test('custom_saved_model_edgetpu.tflite', class_names)

loading term (sec): 2.6942975521087646
images per sec: 188.44121394969542
accuracy: 
{'construction_danger': 0.0, 'construction_safe': 0.0, 'park_danger': 0.0, 'park_safe': 0.0, 'road_danger': 0.0, 'road_safe': 1.0}


/home/keti/tf_2.5.0/lib/python3.6/site-packages/ipykernel_launcher.py:63: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


## custom_resnet_saved_model_edgetpu

In [8]:
test('custom_resnet_saved_model_edgetpu', class_names)

loading term (sec): 0.0021169185638427734
images per sec: 189.95065399835036
accuracy: 
{'construction_danger': 0.0, 'construction_safe': 0.0, 'park_danger': 0.0, 'park_safe': 0.0, 'road_danger': 0.0, 'road_safe': 1.0}


/home/keti/tf_2.5.0/lib/python3.6/site-packages/ipykernel_launcher.py:63: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


## custom_mobilenetv2_saved_model_edgetpu

In [9]:
test('custom_mobilenetv2_saved_model_edgetpu', class_names)

loading term (sec): 0.0011734962463378906
images per sec: 191.13843626551133
accuracy: 
{'construction_danger': 0.0, 'construction_safe': 0.0, 'park_danger': 0.0, 'park_safe': 0.0, 'road_danger': 0.0, 'road_safe': 1.0}


/home/keti/tf_2.5.0/lib/python3.6/site-packages/ipykernel_launcher.py:63: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
